#data
```
{
    "patch": "@@ -53,7 +53,7 @@ public class ProtocGapicPluginGeneratorTest {\n                 model.getFiles().stream().map(ProtoFile::getProto).collect(Collectors.toList()))\n             // Only the file to generate a client for (don't generate dependencies)\n             .addFileToGenerate(\"multiple_services.proto\")\n-            .setParameter(\"language=java\")\n+            .setParameter(\"language=java,transport=grpc\")\n             .build();\n \n     CodeGeneratorResponse response = ProtocGeneratorMain.generate(codeGeneratorRequest);",
    "y": 1,
    "oldf": "/* Copyright 2019 Google LLC\n *\n * Licensed under the Apache License, Version 2.0 (the \"License\");\n * you may not use this file except in compliance with the License.\n * You may obtain a copy of the License at\n *\n *      https://www.apache.org/licenses/LICENSE-2.0\n *\n * Unless required by applicable law or agreed to in writing, software\n * distributed under the License is distributed on an \"AS IS\" BASIS,\n * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n * See the License for the specific language governing permissions and\n * limitations under the License.\n */\npackage com.google.api.codegen.gapic;\n\nimport com.google.api.codegen.CodegenTestUtil;\nimport com.google.api.codegen.ProtocGeneratorMain;\nimport com.google.api.codegen.protoannotations.GapicCodeGeneratorAnnotationsTest;\nimport com.google.api.tools.framework.model.Model;\nimport com.google.api.tools.framework.model.ProtoFile;\nimport com.google.api.tools.framework.model.testing.TestDataLocator;\nimport com.google.common.truth.Truth;\nimport com.google.protobuf.compiler.PluginProtos.CodeGeneratorRequest;\nimport com.google.protobuf.compiler.PluginProtos.CodeGeneratorResponse;\nimport java.util.stream.Collectors;\nimport org.junit.BeforeClass;\nimport org.junit.ClassRule;\nimport org.junit.Test;\nimport org.junit.rules.TemporaryFolder;\n\npublic class ProtocGapicPluginGeneratorTest {\n\n  private static String[] protoFiles = {\"multiple_services.proto\"};\n  private static TestDataLocator testDataLocator;\n  private static Model model;\n  @ClassRule public static TemporaryFolder tempDir = new TemporaryFolder();\n\n  @BeforeClass\n  public static void startUp() {\n    testDataLocator = TestDataLocator.create(GapicCodeGeneratorAnnotationsTest.class);\n    testDataLocator.addTestDataSource(CodegenTestUtil.class, \"testsrc/common\");\n\n    model = CodegenTestUtil.readModel(testDataLocator, tempDir, protoFiles, new String[] {});\n  }\n\n  @Test\n  public void testGenerator() {\n    CodeGeneratorRequest codeGeneratorRequest =\n        CodeGeneratorRequest.newBuilder()\n            // All proto files, including dependencies\n            .addAllProtoFile(\n                model.getFiles().stream().map(ProtoFile::getProto).collect(Collectors.toList()))\n            // Only the file to generate a client for (don't generate dependencies)\n            .addFileToGenerate(\"multiple_services.proto\")\n            .setParameter(\"language=java\")\n            .build();\n\n    CodeGeneratorResponse response = ProtocGeneratorMain.generate(codeGeneratorRequest);\n\n    // TODO(andrealin): Look into setting these up as baseline files.\n    Truth.assertThat(response).isNotNull();\n    Truth.assertThat(response.getError()).isEmpty();\n    Truth.assertThat(response.getFileCount()).isEqualTo(15);\n    Truth.assertThat(response.getFile(0).getContent()).contains(\"DecrementerServiceClient\");\n  }\n\n  @Test\n  public void testFailingGenerator() {\n    CodeGeneratorRequest codeGeneratorRequest =\n        CodeGeneratorRequest.newBuilder()\n            .addAllProtoFile(\n                model.getFiles().stream().map(ProtoFile::getProto).collect(Collectors.toList()))\n            // File does not exist.\n            .addFileToGenerate(\"fuuuuudge.proto\")\n            .build();\n\n    CodeGeneratorResponse response = ProtocGeneratorMain.generate(codeGeneratorRequest);\n\n    Truth.assertThat(response).isNotNull();\n    Truth.assertThat(response.getError()).isNotEmpty();\n  }\n}\n",
    "idx": 1,
    "id": 30879,
    "msg": "can we also test for `transport=rest`?",
    "proj": "googleapis-gapic-generator",
    "lang": "java"
}
```

In [1]:
import os
import sys

import torch.nn as nn
import torch
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss, BCEWithLogitsLoss
import numpy as np
from transformers import (
    RobertaConfig,
    RobertaModel,
    RobertaTokenizer,
    BartConfig,
    BartForConditionalGeneration,
    BartTokenizer,
    T5Config,
    T5ForConditionalGeneration,
    T5Tokenizer,
    AutoTokenizer,
)
import logging
from code.models import ReviewerModel, load_model
import json


mnt_dir="/data/DataLACP/rambo/data/codereviewer"
model_path = os.path.join(mnt_dir, "codereviewer")
max_source_length = 300
max_target_length = 128
mask_rate = 0.15 
beam_size = 10
device = torch.device("cpu")

class CRModel():
    def __init__(self, model_path, max_source_length, max_target_length, mask_rate, beam_size, device):
        self.model_path = model_path
        self.max_source_length = max_source_length
        self.max_target_length = max_target_length
        self.mask_rate = mask_rate
        self.beam_size = beam_size
        self.device = device
        config_class, model_class, tokenizer_class = (T5Config, ReviewerModel, T5Tokenizer)
        config = config_class.from_pretrained(model_path)
        model = model_class.from_pretrained(model_path)
        config, model, tokenizer = load_model(
            config,
            model,
            tokenizer_class,
            add_lang_ids=True,
            tokenizer_path=model_path,
        )

        model_bin_path = os.path.join(model_path, "pytorch_model.bin")
        saved = model.cls_head
        model.cls_head = None
        model.load_state_dict(torch.load(model_bin_path, map_location="cpu"))
        model.cls_head = saved
        #model.to(args.local_rank)
        model.to(device)
        self.model = model
        self.config = config
        self.tokenizer = tokenizer
        
    def predict(self, diff):
        difflines = diff.split("\n")[1:]        # remove start @@
        difflines = [line for line in difflines if len(line.strip()) > 0]
        map_dic = {"-": 0, "+": 1, " ": 2}
        def f(s):
            if s in map_dic:
                return map_dic[s]
            else:
                return 2
        labels = [f(line[0]) for line in difflines]
        difflines = [line[1:].strip() for line in difflines]
        inputstr = ""
        for label, line in zip(labels, difflines):
            if label == 1:
                inputstr += "<add>" + line
            elif label == 0:
                inputstr += "<del>" + line
            else:
                inputstr += "<keep>" + line
        source_ids = self.tokenizer.encode(inputstr, max_length=max_source_length, truncation=True)
        source_ids = source_ids[:max_source_length - 2]
        source_ids = [self.tokenizer.bos_id] + source_ids + [self.tokenizer.eos_id]
        pad_len = max_source_length - len(source_ids)
        source_ids += [self.tokenizer.pad_id] * pad_len

        input_labels = [-100] * len(source_ids)

        pred_ids, ex_ids = [], []
        source_ids = torch.tensor([source_ids], dtype=torch.long).to(device)
        source_mask = source_ids.ne(self.tokenizer.pad_id)
        preds = self.model.generate(source_ids,
                        attention_mask=source_mask,
                        use_cache=True,
                        num_beams=beam_size,
                        early_stopping=True,
                        max_length=max_target_length,
                        )
        top_preds = list(preds.cpu().numpy())
        pred_ids.extend(top_preds)
        pred_nls = [self.tokenizer.decode(id[1:], skip_special_tokens=True, clean_up_tokenization_spaces=False) for id in pred_ids]
        return pred_nls[0]



ModuleNotFoundError: No module named 'code.models'; 'code' is not a package

In [2]:
import json
json_string = '''{
    "patch": "@@ -53,7 +53,7 @@ public class ProtocGapicPluginGeneratorTest {\\n                 model.getFiles().stream().map(ProtoFile::getProto).collect(Collectors.toList()))\\n             // Only the file to generate a client for (don't generate dependencies)\\n             .addFileToGenerate(\\"multiple_services.proto\\")\\n-            .setParameter(\\"language=java\\")\\n+            .setParameter(\\"language=java,transport=grpc\\")\\n             .build();\\n \\n     CodeGeneratorResponse response = ProtocGeneratorMain.generate(codeGeneratorRequest);",
    "y": 1,
    "oldf": "/* Copyright 2019 Google LLC\\n *\\n * Licensed under the Apache License, Version 2.0 (the \\"License\\");\\n * you may not use this file except in compliance with the License.\\n * You may obtain a copy of the License at\\n *\\n *      https://www.apache.org/licenses/LICENSE-2.0\\n *\\n * Unless required by applicable law or agreed to in writing, software\\n * distributed under the License is distributed on an \\"AS IS\\" BASIS,\\n * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\\n * See the License for the specific language governing permissions and\\n * limitations under the License.\\n */\\npackage com.google.api.codegen.gapic;\\n\\nimport com.google.api.codegen.CodegenTestUtil;\\nimport com.google.api.codegen.ProtocGeneratorMain;\\nimport com.google.api.codegen.protoannotations.GapicCodeGeneratorAnnotationsTest;\\nimport com.google.api.tools.framework.model.Model;\\nimport com.google.api.tools.framework.model.ProtoFile;\\nimport com.google.api.tools.framework.model.testing.TestDataLocator;\\nimport com.google.common.truth.Truth;\\nimport com.google.protobuf.compiler.PluginProtos.CodeGeneratorRequest;\\nimport com.google.protobuf.compiler.PluginProtos.CodeGeneratorResponse;\\nimport java.util.stream.Collectors;\\nimport org.junit.BeforeClass;\\nimport org.junit.ClassRule;\\nimport org.junit.Test;\\nimport org.junit.rules.TemporaryFolder;\\n\\npublic class ProtocGapicPluginGeneratorTest {\\n\\n  private static String[] protoFiles = {\\"multiple_services.proto\\"};\\n  private static TestDataLocator testDataLocator;\\n  private static Model model;\\n  @ClassRule public static TemporaryFolder tempDir = new TemporaryFolder();\\n\\n  @BeforeClass\\n  public static void startUp() {\\n    testDataLocator = TestDataLocator.create(GapicCodeGeneratorAnnotationsTest.class);\\n    testDataLocator.addTestDataSource(CodegenTestUtil.class, \\"testsrc/common\\");\\n\\n    model = CodegenTestUtil.readModel(testDataLocator, tempDir, protoFiles, new String[] {});\\n  }\\n\\n  @Test\\n  public void testGenerator() {\\n    CodeGeneratorRequest codeGeneratorRequest =\\n        CodeGeneratorRequest.newBuilder()\\n            // All proto files, including dependencies\\n            .addAllProtoFile(\\n                model.getFiles().stream().map(ProtoFile::getProto).collect(Collectors.toList()))\\n            // Only the file to generate a client for (don't generate dependencies)\\n            .addFileToGenerate(\\"multiple_services.proto\\")\\n            .setParameter(\\"language=java\\")\\n            .build();\\n\\n    CodeGeneratorResponse response = ProtocGeneratorMain.generate(codeGeneratorRequest);\\n\\n    // TODO(andrealin): Look into setting these up as baseline files.\\n    Truth.assertThat(response).isNotNull();\\n    Truth.assertThat(response.getError()).isEmpty();\\n    Truth.assertThat(response.getFileCount()).isEqualTo(15);\\n    Truth.assertThat(response.getFile(0).getContent()).contains(\\"DecrementerServiceClient\\");\\n  }\\n\\n  @Test\\n  public void testFailingGenerator() {\\n    CodeGeneratorRequest codeGeneratorRequest =\\n        CodeGeneratorRequest.newBuilder()\\n            .addAllProtoFile(\\n                model.getFiles().stream().map(ProtoFile::getProto).collect(Collectors.toList()))\\n            // File does not exist.\\n            .addFileToGenerate(\\"fuuuuudge.proto\\")\\n            .build();\\n\\n    CodeGeneratorResponse response = ProtocGeneratorMain.generate(codeGeneratorRequest);\\n\\n    Truth.assertThat(response).isNotNull();\\n    Truth.assertThat(response.getError()).isNotEmpty();\\n  }\\n}\\n",
    "idx": 1,
    "id": 30879,
    "msg": "can we also test for `transport=rest`?",
    "proj": "googleapis-gapic-generator",
    "lang": "java"
}'''

example = {"msg":"",
           "patch":'''"@@ -182,7 +182,9 @@ abstract class AbstractSolrBackendFactory implements FactoryInterface\n      */\n     protected function createBackend(Connector $connector)\n     {\n+        $config = $this->config->get($this->mainConfig);\n         $backend = new $this->backendClass($connector);\n+        $backend->setPageSize($config->Index->record_batch_size);\n         $backend->setQueryBuilder($this->createQueryBuilder());\n         $backend->setSimilarBuilder($this->createSimilarBuilder());\n         if ($this->logger) {",
    '''}

diff = example["patch"]
model = CRModel(model_path, max_source_length, max_target_length, mask_rate, beam_size, device)
model.predict(diff)

<class 'transformers.models.t5.tokenization_t5.T5Tokenizer'> <class 'str'>


/home/rambo/miniconda3/envs/spaco/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/tmp/ipykernel_2984665/3700254775.py:56: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to 

'`$config->Index->record_batch_size` -> `$config->IndexBatchSize`'